Para correr el programa, apreta el botón para correr todas las celdas, por favor se paciente, funciona :) , luego puedes utilizar las funciones de forma casi instantánea.

In [ ]:
print('Por favor espera unos minutos mientras se carga el archivo...')

In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
from IPython.display import display_html
import warnings
warnings.filterwarnings("ignore")

raw_tweets = pd.read_json(r'farmers-protest-tweets-2021-03-5.json', lines=True)
raw_tweets = raw_tweets[raw_tweets['lang']=='en']

In [ ]:
users = json_normalize(raw_tweets['user'])
users.drop(['description', 'linkTcourl'], axis=1, inplace=True)
users.rename(columns={'id':'userId', 'url':'profileUrl'}, inplace=True)

user_id = []
for user in raw_tweets['user']:
    uid = user['id']
    user_id.append(uid)
raw_tweets['userId'] = user_id

In [ ]:
cols = ['id', 'retweetCount', 'userId', 'url', 'date', 'renderedContent']
tweets = raw_tweets[cols]
tweets.rename(columns={'id':'tweetId', 'url':'tweetUrl'}, inplace=True)

tweets = pd.DataFrame(tweets)
tweets.drop_duplicates(subset=['tweetId'], inplace=True)

In [ ]:
top_tweets = tweets.nlargest(10, 'retweetCount')

In [ ]:
users_tweets = tweets[['tweetId', 'tweetUrl','renderedContent', 'userId']].merge(users[['userId','displayname']], on='userId')
top_users = users_tweets.groupby(['displayname']).size().reset_index(name='tweets').sort_values(by='tweets', ascending=False)

In [ ]:
tweets['date'] = tweets['date'].dt.strftime('%m-%d-%Y')

In [ ]:
top_days = tweets.groupby(['date']).size().reset_index(name='tweets').sort_values(by='tweets', ascending=False)

In [ ]:
top_hashtags = tweets.renderedContent.str.extractall(r'(\#\w+)')[0].value_counts()
top_hashtags.sort_values(ascending=False)

In [ ]:
def show_table(df):
  display_html(df.head(10))

In [ ]:
def menu():
  print('\nArchivo cargado')
  exit_program = False
  while not exit_program:
    print(
      '''
      Elige el número de la función a ejecutar:
      [ 1 ] : Top 10 tweets
      [ 2 ] : Top 10 users
      [ 3 ] : Top 10 days
      [ 4 ] : Top 10 hashtags
      [ 5 ] : Salir del programa
      '''
    )
    input_correct = False
    while not input_correct:
      user_input = input('Ingresa un número: ')
      try:
        value = int(user_input)
        if value < 1 or value > 5:
          print('Error: Ingresa un número entre 1 y 5')
        else:
          input_correct = True
          if value == 1:
            print('\nTop 10 tweets:')
            show_table(top_tweets)
          elif value == 2:
            print('\nTop 10 users:')
            show_table(top_users)
          elif value == 3:
            print('\nTop 10 days:')
            show_table(top_days)
          elif value == 4:
            print('\nTop 10 hashtags:\n')
            print(top_hashtags.head(10))
          elif value == 5:
            print('Programa finalizado.')
            exit_program = True
      except ValueError:
        print('Error: Recuerda ingresar un número')
        continue

In [ ]:
menu()